## Описание

Домашнее задание номер 2. Про обман сеток.

Идея заключается в том, обученную сеть можно обмануть подсунув ей специальную картинку. Обмануть в данном случае означает, что сетка на картинке перестанет видеть то что дожна. 
Чтобы так сделать надо будет медленно менять картинку которую сеть классифицирует правильно в сторону неправильной классификации. 

In [8]:
import numpy as np
import torch
import matplotlib.pyplot as plt

In [9]:
from IPython import display

%matplotlib inline

In [10]:
import torch.nn as nn
import torch.nn.functional as F

### Загружаем данные 

In [4]:
import pandas as pd

data = pd.read_csv("../../6.Intro_to_NN/data/train.csv")

x = data[data.columns[1:]].values
y = data[data.columns[0]].values

In [7]:
x = x.reshape(-1, 28,28)

In [5]:
from sklearn.model_selection import train_test_split

x_tr, x_te, y_tr, y_te = train_test_split(x, y)

Будьте внимательны, если вы используете GPU, все тензоры должны быть так или иначе отправленны туда. Например с помощью метода .cuda()

In [ ]:
x_tr = torch.from_numpy(x_tr.astype(np.float32)).cuda()
x_te = torch.from_numpy(x_te.astype(np.float32)).cuda()
y_tr = torch.from_numpy(y_tr.astype(int)).cuda()
y_te = torch.from_numpy(y_te.astype(int)).cuda()

In [ ]:
x_tr = x_tr.reshape(x_tr.shape[0], 1, x_tr.shape[1], x_tr.shape[2])
x_te = x_te.reshape(x_te.shape[0], 1, x_te.shape[1], x_te.shape[2])

Код для загрузки батчей

In [ ]:
def iterate_minibatches(x, y, batchsize, shuffle=True):
    if shuffle:
        indices = np.arange(x.shape[0])
        np.random.shuffle(indices)
    for start_idx in range(0, x.shape[0] - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield x[excerpt], y[excerpt]

## Инициализируем сетку

Подготовьте произвольную сверточную сетку для классификации всех 10-ти цифр. Можно использовать все что мы обсуждали ранее. 

Важно, чтобы сетка умела возвращать не только окончательные ответы после софтмакса, а еще результат до применения софтмакса (называется logit). 

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        
        *YOUR CODE HERE*
        
    def get_logits(self, x):
        
        *ФУНКЦИЯ ПРОГОНЯЕТ х ЧЕРЕЗ ВСЮ СЕТКУ, НЕ ПРИМЕНЯЕТ ТОЛЬКО СОФТМАКС В КОНЦЕ*
        *YOUR CODE HERE*
        
       
        
    def forward(self, x):
        return F.softmax(self.get_logits(x), dim = 1)


net = Net()#.cuda()
print(net)

## Обучение

Ниже стандартный код для обучения. Обучите сетку на всем датасете

In [ ]:
from torch.optim import Adam, SGD

In [ ]:
criterion = *YOUR CODE HERE*
sgd = *YOUR CODE HERE*

In [ ]:
train_losses = []
val_losses = []

In [ ]:
epochs = 50

In [ ]:
for epoch in range(epochs):
    batch_losses = []
    for i, (batch_x, batch_y) in enumerate(iterate_minibatches(x_tr[:], y_tr[:], 20)):       
        out = net(batch_x)
        target = batch_y

        sgd.zero_grad()
        loss = criterion(out, target)

        loss.backward()
        sgd.step()
        
        batch_losses.append(loss.item())

    train_losses.append(np.mean(batch_losses))
    
    val_losses.append(criterion(net(x_te), y_te).item())

    
    display.clear_output(wait=True)
    plt.plot(train_losses, label='train')
    plt.plot(val_losses, label='val')
    plt.legend()
    plt.show()

## Теперь начнем обман.

Выберете произвольный пример из тестовой выборки, который сетка достаточно увернено и правильно классифицирует. И включите градиенты для этой картинки 

In [ ]:
example_image = *YOUR CODE HERE*
*YOUR CODE HERE*

Давайте поставим себе задачу сделать так, чтобы та же самая сетка эту картинку классифицировала неверно. Выберите произвольный другой ответ для этой цифры. Напимер если была восьмерка, давайте заствим сеть думать, что это четверка. 

Теперь будем менять картинку так чтобы сетка начала ошибаться. Для этого надо будет посчитать производную выходного нейрона отвечающего за желаемую цифру (четверку) по исходной картинке, а дальше немного обновить картинку в сторону увеличения этого выходного нейрона (четверки).

Сначала забросим картинку в сеть и выведем логиты, а не ответ, чтобы софтмакс не убил градиенты.

In [ ]:
logits = net.get_logits(example_image)

Теперь нужно сделать logits.backward(), но просто так он не заработает, потому что logits это вектор и backward() не будет знать производные какого из элементов хочется посчитать. 

Для таких ситуаций у backward() можно передать на вход тензор такой же формы, как тот от которого bacward вызвана c коэффициентами. И итоговые градиенты будут суммой градиентов от кадого элемента тензора от которого вызвана backward, умноженными на соответствующие коэффициенты. 

Пример:

In [57]:
x = torch.ones((2,2), requires_grad=True)
y = torch.rand((2,2), requires_grad=True)

In [58]:
print(x)
print(y)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[0.1841, 0.3100],
        [0.1226, 0.4917]], requires_grad=True)


In [59]:
z = x.mm(y)
print(z)
z.backward(torch.ones(2,2))

print(x.grad)
print(y.grad)

tensor([[0.3067, 0.8016],
        [0.3067, 0.8016]], grad_fn=<MmBackward>)
tensor([[0.4941, 0.6143],
        [0.4941, 0.6143]])
tensor([[2., 2.],
        [2., 2.]])


In [60]:
x.grad.zero_()
y.grad.zero_()

tensor([[0., 0.],
        [0., 0.]])

In [61]:
z = x.mm(y)
print(z)
z.backward(torch.from_numpy(np.array([[1.,0.], [0., 0.]]).astype(np.float32)))

print(x.grad)
print(y.grad)

tensor([[0.3067, 0.8016],
        [0.3067, 0.8016]], grad_fn=<MmBackward>)
tensor([[0.1841, 0.1226],
        [0.0000, 0.0000]])
tensor([[1., 0.],
        [1., 0.]])


Соответственно, чтобы нам посчитать градиенты нужного выходного нейрона по картинке нужно вызвать logits.backward() и передать туда тензор такой же формы как logits, такой чтобы остались только нужные производные.

In [ ]:
logits.backward(*YOUR CODE HERE*)

Теперь осталось обновлять картинку в цикле, пока сеть не начнет думать, что это новая цифра с вероятность 70-80%

In [ ]:
while *YOUR CODE HERE*:
    logits = net.get_logits(example_image)
    logits.backward(*YOUR CODE HERE*)
    
    example_image.data = *YOUR CODE HERE*
    example_image.grad.zero_()
    
    

Теперь нарисуйте получившееся изображение используя plt.imshow() и присылайте в чат.